# Setup

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

from torchmetrics import Accuracy
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR

from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from sklearn import metrics

In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import random
from glob import glob
from tqdm import tqdm
from scipy.io import loadmat

import torch
from torch import nn
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

from datetime import datetime

In [3]:
from IPython.display import clear_output
import os, sys, shutil
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from tqdm import tqdm, trange
from glob import glob
import random
import cv2 as cv
import torch.nn.functional as F
import json
from sklearn.preprocessing import LabelEncoder

In [4]:
# main_data_dir =  "/media/mountHDD2"
print(os.getcwd())
save_dir = os.getcwd()
os.chdir("..")
# os.chdir("..")
main_dir = os.getcwd() 
print(os.listdir(main_dir))
print(save_dir)

/media/mountHDD2/thao/git/HeartResearch/Chapman Dataset/Reimplement
['Reimplement', 'Data_Explore', 'Diagnostics.xlsx', '.ipynb_checkpoints', '2D_model', '1D_Model', '2D_Loss']
/media/mountHDD2/thao/git/HeartResearch/Chapman Dataset/Reimplement


In [5]:
data_dir = "/media/mountHDD3/data_storage/biomedical_data/ecg_data/ECGDataDenoised"
label_file = main_dir + "/Diagnostics.xlsx"

In [6]:
ls = []
for file in glob(data_dir + "/*"):
    ls.append(file.split("/")[-1].split(".")[0].split("_")[0])
ls_list = pd.Series(ls)
ls_list.value_counts()

MUSE    10588
dtype: int64

In [7]:
diag_df = pd.read_excel(label_file)
label_df = diag_df[['FileName', 'Rhythm']]
label_df

,FileName,Rhythm
0,MUSE_20180120_121711_19000,SA
1,MUSE_20180120_121704_86000,SA
2,MUSE_20180113_125357_13000,SA
3,MUSE_20180113_134825_04000,SB
4,MUSE_20180115_123455_79000,SB
...,...,...
10641,MUSE_20181222_204246_47000,SVT
10642,MUSE_20180115_120332_79000,SA
10643,MUSE_20180712_152507_30000,AF
10644,MUSE_20180118_181350_17000,SA


In [8]:
# unique_values = np.unique(label_df["Rhythm"].values.tolist())
# print(unique_values)

In [9]:
data_paths = []
for file in glob(data_dir +"/*"):
    data_paths.append(file)

In [10]:
# data_dict = {
#     idx : [] for idx in unique_values
# }

# for data_path in data_paths:
#     filename = data_path.split("/")[-1].split(".")[0]
    
#     # Get the corresponding 'Rhythm' values for the filename
#     rhythm_values = label_df[label_df["FileName"] == filename]["Rhythm"].values
    
#     if rhythm_values.size == 1:
#         _cls = rhythm_values.item()  # Safe to use .item() if there's exactly one match
#     elif rhythm_values.size > 1:
#         _cls = rhythm_values[0]  # Take the first element if there are multiple matches
#     else:
#         continue  # Skip if no matches are found
    
#     data_dict[_cls].append(data_path)

# for key in data_dict:
#     print(f"{key}->{len(data_dict[key])}")

In [11]:
label_encoder = LabelEncoder()
label_df['Rhythm'] = label_encoder.fit_transform(label_df['Rhythm'])
print(label_df)

                         FileName  Rhythm
0      MUSE_20180120_121711_19000       5
1      MUSE_20180120_121704_86000       5
2      MUSE_20180113_125357_13000       5
3      MUSE_20180113_134825_04000       7
4      MUSE_20180115_123455_79000       7
...                           ...     ...
10641  MUSE_20181222_204246_47000      10
10642  MUSE_20180115_120332_79000       5
10643  MUSE_20180712_152507_30000       0
10644  MUSE_20180118_181350_17000       5
10645  MUSE_20180116_121646_28000       9

[10646 rows x 2 columns]


/tmp/ipykernel_3482338/787631178.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  label_df['Rhythm'] = label_encoder.fit_transform(label_df['Rhythm'])


In [12]:
filenames = label_df["FileName"].values.tolist()

In [13]:
unique_values = np.unique(label_df["Rhythm"].values.tolist())
print(unique_values)

[ 0  1  2  3  4  5  6  7  8  9 10]


In [14]:
data_paths = []
for file in glob(data_dir +"/*"):
    data_paths.append(file)
print(len(data_paths))

10588


In [15]:
print(type(label_df["Rhythm"][0]))

<class 'numpy.int64'>


In [16]:
ratio = [0.8, 0.1]

train_index = int(len(data_paths)*ratio[0])
valid_index = int(len(data_paths)*(ratio[0]+ratio[1]))
print(train_index)
train_mat_paths = data_paths[:train_index]
valid_mat_paths = data_paths[train_index:valid_index]
print(len(train_mat_paths))

train_label = label_df.iloc[:train_index,:]
valid_label = label_df.iloc[train_index:valid_index,:]

8470
8470


In [17]:
print(len(train_label))
print(len(valid_label))

8470
1059


In [18]:
label_df[label_df["FileName"] == filenames]["Rhythm"].values

array([5, 5, 5, ..., 0, 5, 9])

 # Data Loader

In [19]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torch.nn.functional import one_hot

In [20]:
# data = []
class HeartData(Dataset):
    def __init__(self, data_paths, label_df):
        self.data_paths = data_paths
        random.shuffle(self.data_paths)
        self.label_df = label_df
        normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
        self.transform = transforms.Compose([
            transforms.Resize((224, 224), antialias=None),
            normalize
        ])

    def __getitem__(self, idx):
        data_path = self.data_paths[idx]        
        # data = loadmat(data_path)['ECG'][0][0][2]
        data = pd.read_csv(data_path, header = None)
        data = data.values.T
        clip_data = data[:, 500:3000]
        clip_data = torch.tensor(clip_data, dtype=torch.float32)
        # normalized_data = (clip_data - clip_data.min()) / (clip_data.max() - clip_data.min())
        # grayscale_images = (normalized_data * 255)
        # grayscale_images = grayscale_images.unsqueeze(0).unsqueeze(0) # (1, 1, h, w)
        # resized_images = F.interpolate(grayscale_images, size=(9*4,2500), mode='bilinear', align_corners=True)
        # resized_images = resized_images.squeeze(0).squeeze(0)
        # torch_data = resized_images.unsqueeze(0).repeat(3, 1, 1)
        # torch_data_resize = self.transform(torch_data)

        filename = data_path.split("/")[-1].split(".")[0]
        label = self.label_df[self.label_df["FileName"] == filename]["Rhythm"].values.item()

        return clip_data, label

    def __len__(self):
        return len(self.data_paths)    

In [21]:
train_ds = HeartData(train_mat_paths, label_df)
valid_ds = HeartData(valid_mat_paths, label_df)

In [22]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu", index = 1)
batch_size = 1

traindl = DataLoader(
    train_ds,
    batch_size=batch_size, 
    shuffle=True, 
    pin_memory=True, 
    num_workers=os.cpu_count()//2
)

validdl = DataLoader(
    valid_ds,
    batch_size=1, 
    # shuffle=True, 
    pin_memory=True, 
    num_workers=os.cpu_count()//2
)

# Model 

In [23]:
class Dilatex12(nn.Module):
    def __init__(self, in_channel):
        super(Dilatex12, self).__init__()
        #input_size(1,1,300) 
        self.in_channel = in_channel

        #Dilation block
        self.dilate1 = nn.Conv1d(self.in_channel, 6, kernel_size = 5, stride=1, dilation=1, padding = 0) # 296
        self.dilate2 = nn.Conv1d(self.in_channel, 6, kernel_size = 5, stride=1, dilation=2, padding = 2) 
        self.dilate3 = nn.Conv1d(self.in_channel, 6, kernel_size = 5, stride=1, dilation=3, padding= 4) 
        self.dilate4 = nn.Conv1d(self.in_channel, 6, kernel_size = 5, stride=1, dilation=4, padding= 6) 

        self.conv1 = nn.Sequential(
            nn.BatchNorm1d(24),
            nn.ReLU(),
            nn.MaxPool1d(3, stride = 2)
        )
                
        # FIRST Conv1D layer (on the left branch)
        self.conv1d_left = nn.Conv1d(24, out_channels=16, kernel_size=6, stride=1)
        
        # FIRST Conv1D layer (right branch)
        self.conv1d_right_1 = nn.Sequential(
            nn.Conv1d(24, out_channels=8, kernel_size=3, stride=1), 
            nn.BatchNorm1d(8),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=1),
        )
        self.conv1d_right_2 = nn.Sequential(
            nn.Conv1d(in_channels=8, out_channels=16, kernel_size=3, stride=1),
            nn.BatchNorm1d(16)
        )
        self.add = nn.Sequential(
            nn.ReLU(),
            nn.MaxPool1d(2, stride = 1)
        )

        # SECOND Conv1D (left)
        self.conv1d_left2 = nn.Conv1d(16, out_channels=16, kernel_size=7, stride=1)
        
        # SECOND Conv1D layer (right)
        self.conv1d_right2_1 = nn.Sequential(
            nn.Conv1d(16, out_channels=16, kernel_size=3, stride=1), 
            nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=3, stride=1),
        )
        self.conv1d_right2_2 = nn.Sequential(
            nn.Conv1d(in_channels=16, out_channels=16, kernel_size=3, stride=1),
            nn.BatchNorm1d(16)
        )

        # 2 LAYERS BI-LSTM
        self.lstm1 = nn.LSTM(16, 12, bidirectional=True, num_layers=2, batch_first=True)
        self.tanh = nn.Tanh() 
        self.batch1 = nn.BatchNorm1d(24)
        self.drop1 = nn.Dropout(0.01)
        self.lstm2 = nn.LSTM(24, 24, bidirectional=True, num_layers=2, batch_first=True)
        self.batch2 = nn.BatchNorm1d(48)
        self.drop2 = nn.Dropout(0.02)

    def forward(self, x):
        out_list = []
        a = torch.tensor_split(x, 12, dim = 1)
        for x_lead in a:
            # x_lead = x_lead.to(device)
            # x1 = self.dilate1(x_lead)
            # # print(x1.size())
            # x2 = self.dilate2(x_lead)
            # # print(x2.size())
            # x3 = self.dilate3(x_lead)
            # # print(x3.size())
            # x4 = self.dilate4(x_lead)
            # print(x4.size())
            x5 = torch.cat((self.dilate1(x_lead),self.dilate2(x_lead),self.dilate3(x_lead),self.dilate4(x_lead)),1)
            # print(x5.size())
            out_block1 = self.conv1(x5)
            # print(out_block1.size())
            
            y1 = self.conv1d_left(out_block1)
            # print(y1.size())
            y2 = self.conv1d_right_1(out_block1)
            y2 = self.conv1d_right_2(y2)
            # print(y2.size())
            out_add1 = torch.add(y1,y2)
            # print(out_add1.size())
    
            z1 = self.conv1d_left2(out_add1)
            z2 = self.conv1d_right2_1(out_add1)
            z2 = self.conv1d_right2_2(z2)
            out_add2 = torch.add(z1,z2)
            # print(out_add2.size())
    
            out_add2 = out_add2.permute(0, 2, 1)
    
            lstm1, _ = self.lstm1(out_add2)
            lstm1 = self.tanh(lstm1)
            lstm1 = lstm1.permute(0, 2, 1)
            lstm1 = self.batch1(lstm1)
            lstm1 = lstm1.permute(0, 2, 1)
            lstm1 = self.drop1(lstm1)
            
            out, _ = self.lstm2(lstm1)
            out = self.tanh(out)
            out = out.permute(0, 2, 1)
            out = self.batch2(out)
            out = out.permute(0, 2, 1)
            out = self.drop2(out)
            out_list.append(out.detach().to(device))
        output = torch.cat(out_list,1) 
        return output

In [24]:
# a = torch.rand(1,12,5000)
# # a = torch.tensor_split(s, 12, dim = 1)
# # print(type(a))
# dilate = Dilatex12(1)(a)
model1 = Dilatex12(1)
# print(dilate.shape)

In [25]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class AttentionLayer(nn.Module):
    def __init__(self, feature_dim):
        super(AttentionLayer, self).__init__()
        self.W = nn.Parameter(torch.randn(feature_dim, feature_dim))  # Weight matrix
        self.b = nn.Parameter(torch.zeros(1, 1, feature_dim))         # Bias vector

    def forward(self, H):
        # Step 1: Compute Q
        Q = torch.tanh(torch.matmul(H, self.W) + self.b)  # Shape: (batch_size, sequence_length, feature_dim)

        # Step 2: Compute A (attention logits)
        A = torch.bmm(Q, H.transpose(1, 2))  # Shape: (batch_size, sequence_length, sequence_length)

        # Step 3: Compute W_attn (attention weights)
        W_attn = F.softmax(A, dim=-1)  # Shape: (batch_size, sequence_length, sequence_length)

        # Step 4: Compute Yattn (attention-weighted output)
        Yattn = torch.bmm(W_attn, H)  # Shape: (batch_size, sequence_length, feature_dim)

        return Yattn

# batch_size, sequence_length, feature_dim = 1, dilate.shape[1], 48
# attention_layer = AttentionLayer(feature_dim=48)
model2 = AttentionLayer(feature_dim=48)
# Yattn = attention_layer(dilate)
# print(Yattn.shape)

In [26]:
import torch
import torch.nn as nn

class ClassificationNetwork(nn.Module):
    def __init__(self, num_classes):
        super(ClassificationNetwork, self).__init__()
        self.conv2D = nn.Sequential(
            nn.Conv2d(1, out_channels=64, kernel_size=(1, 12), stride=(1, 1)), 
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((1, 1))
        )

        self.dense1 = nn.Sequential(
            nn.Linear(64, 128),
            nn.LayerNorm(128),  # Corrected to BatchNorm1d
            nn.ReLU(),
            nn.Dropout(0.1)
        )
        self.dense2 = nn.Sequential(
            nn.Linear(128, 64),
            nn.LayerNorm(64),  # Corrected to BatchNorm1d
            nn.ReLU(),
            nn.Dropout(0.15)
        )
        
        self.output_layer = nn.Linear(64, num_classes)

    def forward(self, x):
        # x shape: [batch_size, seq_len, features]
        x = x.unsqueeze(1)  # Add channel dimension, shape: [batch_size, 1, seq_len, features]
        x = self.conv2D(x)  # Output shape: [batch_size, 64, 1, 1]
        x = x.view(x.size(0), -1)  # Flatten, shape: [batch_size, 64]
        
        # First Dense Block
        x = self.dense1(x)    
        # Second Dense Block
        x = self.dense2(x)
        
        # Output Layer
        x = self.output_layer(x)
        if self.output_layer.out_features == 1:
            x = torch.sigmoid(x)  # For binary classification
        else:
            x = torch.softmax(x, dim=1)  # For multi-class classification

        return x

In [27]:
model = ClassificationNetwork(num_classes=11)
# output = model(Yattn)
# print(output)

In [28]:
# from torchinfo import summary 
# summary(model, input_size=(1,1236,48))

In [29]:
# from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
# from torch import nn

# class HeartModel(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.ori_model = efficientnet_b0(weights = EfficientNet_B0_Weights.IMAGENET1K_V1)
#         self.ori_model.classifier = nn.Sequential(
#             nn.Dropout(0.2),
#             nn.Linear(1280, 11),
#             # nn.Softmax(dim = 1)
#         )
        
#     def forward(self, x):        
#         x = self.ori_model(x)
#         return x
# model = HeartModel()
# x = torch.randn((1, 3, 224, 224))
# y = torch.randint(0, 10, (1,))
# print(y.shape)
# out = model(x)
# print(nn.CrossEntropyLoss()(out, y))

# Training

In [30]:
epoch = 150
lr = 0.0005 # lr = 0.001 Acc: 0.821875 lr = 0.0005 Acc: 0.825 Focal_loss Acc: 0.828
best_acc = 0
best_ep = 0

model.to(device)
model1.to(device)
model2.to(device)

# optimizer = Adam(model.parameters(), lr=lr)
optimizer = Adam([
    {'params': model1.lstm1.parameters(), 'weight_decay': 0.009},
    {'params': model1.lstm2.parameters(), 'weight_decay': 0.01},
    {'params': model.dense1.parameters(), 'weight_decay': 0.005},
    {'params': model.dense2.parameters(), 'weight_decay': 0.009},
    {'params': model.output_layer.parameters(), 'weight_decay': 0.0}  # No regularization
], lr=0.001)
scheduler = CosineAnnealingLR(optimizer=optimizer, T_max=epoch*len(traindl))

In [31]:
class FocalClassifierV0(nn.Module):
    def __init__(self, gamma=0.3):
        super().__init__()
        
        self.gamma = gamma
        self.act = nn.LogSoftmax(dim=1)
    
    def forward(self, pred, target):

        logits = self.act(pred)

        B, C = tuple(logits.size())

        entropy = torch.pow(1 - logits, self.gamma) * logits * F.one_hot(target, num_classes=C).float()

        return (-1 / B) * torch.sum(entropy)

focalloss_fn = FocalClassifierV0()
loss_fn = nn.CrossEntropyLoss()
# checkpoint_folder = "run_efficientB0_heatmap_gamma0.5_lr0.0005"
checkpoint_folder = "run_attention_gamma0.3_0.01_lr00005"
# checkpoint_folder = "run_proposed_gamma5_0.01_lr0001_10"

In [32]:
def loss_plot(train_losses, val_losses, n_epochs, check_folder):
    now = datetime.now().strftime("%m-%d-%Y - %H-%M-%S")
    run_dir = save_dir + f"/{check_folder}"
    if not os.path.exists(run_dir):
        os.mkdir(run_dir)
    save_loss_dir = run_dir + "/save_losses"
    if not os.path.exists(save_loss_dir):
        os.mkdir(save_loss_dir)
    save_fig_losses = os.path.join(save_loss_dir, f"plot_losses_epoch{n_epochs}_{now}.png")  

    plt.figure()
    plt.plot(range(epoch), np.array(train_losses), label='Train Loss')
    plt.plot(range(epoch), np.array(val_losses), label='Test Loss')
    plt.xlabel('Epoch')
    # plt.xticks()
    plt.ylabel('Loss')
    plt.title('Training and Test Loss')
    plt.legend()
    plt.savefig(save_fig_losses)
    

def acc_plot(train_cls_acc, val_cls_acc, n_epochs, check_folder):
    now = datetime.now().strftime("%m-%d-%Y - %H-%M-%S")
    run_dir = save_dir + f"/{check_folder}"
    if not os.path.exists(run_dir):
        os.mkdir(run_dir)
#         save_dir = run_dir + f"/{now}"
#         if not os.path.exists(save_dir):
#             os.mkdir(save_dir)
    save_acc_dir = run_dir + "/save_acc"
    if not os.path.exists(save_acc_dir):
        os.mkdir(save_acc_dir)
    save_fig_acc = os.path.join(save_acc_dir, 'plot_acc_epoch{}_{}.png'.format(n_epochs, now))  

    plt.figure()
    plt.plot(range(epoch), np.array(train_cls_acc), label='Train Accuracy')
    plt.plot(range(epoch), np.array(val_cls_acc), label='Test Accuracy')
    plt.xlabel('Epoch')
    # plt.xticks
    plt.ylabel('Accuracy')
    plt.title('Training and Test Accuracy')
    plt.legend()
    plt.savefig(save_fig_acc)

def checkpoint(valid_class_acc, 
               val_total_loss,
               old_valid_class_acc,
               old_valid_loss,
               epoch, 
               model,
               optimizer,
               check_folder
#                    logs
              ):

    if valid_class_acc >= old_valid_class_acc and val_total_loss <= old_valid_loss:
        old_valid_class_acc = valid_class_acc
        old_valid_loss = val_total_loss
        save_dict = {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': val_total_loss,
            'test_acc': valid_class_acc
        }

     # Saving best model
        now = datetime.now().strftime("%m-%d-%Y - %H-%M-%S")
        run_dir = save_dir + f"/{check_folder}"
        if not os.path.exists(run_dir):
            os.mkdir(run_dir)
    #         save_dir = run_dir + f"/{now}"
    #         if not os.path.exists(save_dir):
    #             os.mkdir(save_dir)
        save_best_model_dir = run_dir + "/save_best_model"
        if not os.path.exists(save_best_model_dir):
            os.mkdir(save_best_model_dir)
        save_best_model_path = save_best_model_dir + f"/{save_dict['loss']:>7f}_{save_dict['test_acc']:>7f}_{now}.pt"
        torch.save(save_dict, save_best_model_path)
        
def classification_report_csv(report, auc, check_folder):
    now = datetime.now().strftime("%m-%d-%Y - %H-%M-%S")
    run_dir = save_dir + f"/{check_folder}"
    if not os.path.exists(run_dir):
        os.mkdir(run_dir)
    save_report_dir = run_dir + "/save_classification_report"
    if not os.path.exists(save_report_dir):
        os.mkdir(save_report_dir)
        
    report_data = report['macro avg']
    del report_data['support']
    report_data.update({'auc': auc})
#     print(type(report_data))
#     print(report_data)

#     dataframe = pd.DataFrame.from_dict(report_data, orient='index')
#     save_report_file = save_report_dir + f"/classification_report_{now}.csv"
#     dataframe.to_csv(save_report_file, index = False)
    with open(save_report_dir + f"/cls_report_{now}.json", "w") as outfile: 
        json.dump(report_data, outfile)
    
def acc_loss_json(log_dict, check_folder):
    now = datetime.now().strftime("%m-%d-%Y - %H-%M-%S")
    run_dir = save_dir + f"/{check_folder}"
    if not os.path.exists(run_dir):
        os.mkdir(run_dir)
    save_json_dir = run_dir + "/save_acc_loss_json"
    if not os.path.exists(save_json_dir):
        os.mkdir(save_json_dir) 
    save_acc_loss_file = save_json_dir + f"/acc_loss_{now}.json"
    with open(save_acc_loss_file, "w") as outfile: 
        json.dump(log_dict, outfile)

In [33]:
log_dict = {
    "train": {
        "acc": [],
        "loss": []
    },
    "valid": {
        "acc": [],
        "loss": []
    }
}

In [34]:
# Training

# class_la = np.unique(label_df["Rhythm"].values.tolist())
class_la = []
for i in range (11):
    class_la.append(i)
for i in range (len(class_la)):
    class_la[i] = str(class_la[i])
train_losses = []
train_acc_plot = []
val_losses = []
val_acc_plot = []
old_valid_class_acc = 0
old_valid_loss = 1e23
for e in range(epoch):
    model1.train()
    model2.train()
    model.train()
    print(f"Epoch: {e+1}")
    batch_cnt = 0
    total_loss = 0
    correct = 0
    for batch, (train_sig, train_label) in tqdm(enumerate(traindl)):
        batch_cnt = batch
        train_sig = train_sig.to(device)
        # print(train_sig.shape)
        train_label = train_label.to(device)

        out1 = model1(train_sig)
        out2 = model2(out1)
        pred = model(out2)

        loss = loss_fn(pred, train_label)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        scheduler.step()
        total_loss += loss.item()
        correct += (pred.argmax(1) == train_label).type(torch.float).sum().item()
        
    total_loss /= batch_cnt
    correct /= len(traindl.dataset)
    log_dict["train"]["loss"].append(total_loss)
    log_dict["train"]["acc"].append(correct)
    
    print(f"train loss: {total_loss} - train acc: {100*correct}")

# Valid
    batch_cnt = 0
    val_total_loss = 0
    val_correct = 0
    model1.eval()
    model2.eval()
    model.eval()
    y_true_list = [] 
    pred_list = []
    with torch.no_grad():
        for batch, (valid_sig, valid_label) in tqdm(enumerate(validdl)):
            batch_cnt = batch
            valid_label = valid_label.to(device)
            
            valid_sig = valid_sig.to(device)  
            out1 = model1(valid_sig)
            out2 = model2(out1)
            pred = model(out2)
            pred_pos = pred.argmax(1)
            
            y_true_list.append(valid_label)
            pred_list.append(pred_pos)
            
            loss = loss_fn(pred, valid_label)
            val_total_loss += loss.item()
            val_correct += (pred.argmax(1) == valid_label).type(torch.float).sum().item()
            
        val_total_loss /= batch_cnt
        val_correct /= len(validdl.dataset)
        log_dict["valid"]["loss"].append(val_total_loss)
        log_dict["valid"]["acc"].append(val_correct)

        if val_correct > best_acc:
            best_acc = val_correct
            best_ep = e 

        print(f"valid loss: {val_total_loss} - valid acc: {100*val_correct}")
#         checkpoint(valid_class_acc = val_correct, 
#                    val_total_loss = val_total_loss,
#                    old_valid_class_acc = old_valid_class_acc ,
#                    old_valid_loss = old_valid_loss,
#                    epoch = e, 
#                    model = model,
#                    optimizer = optimizer,
#                    check_folder = checkpoint_folder)
        if val_correct >= old_valid_class_acc:
            old_valid_class_acc = val_correct
            old_valid_loss = val_total_loss
            save_dict = {
                'epoch': e,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': val_total_loss,
                'test_acc': val_correct
            }
         # Saving best model
            now = datetime.now().strftime("%m-%d-%Y - %H-%M-%S")
            run_dir = save_dir + f"/{checkpoint_folder}"
            if not os.path.exists(run_dir):
                os.mkdir(run_dir)
            save_best_model_dir = run_dir + "/save_best_model"
            if not os.path.exists(save_best_model_dir):
                os.mkdir(save_best_model_dir)
            save_best_model_path = save_best_model_dir + f"/{save_dict['loss']:>7f}_{save_dict['test_acc']:>7f}_{now}.pt"
            torch.save(save_dict, save_best_model_path)
        
print(f"Best acuracy: {best_acc} at epoch {best_ep}")

y_true = torch.cat(y_true_list).cpu().numpy()
pred = torch.cat(pred_list).cpu().numpy()

acc_loss_json(log_dict, check_folder = checkpoint_folder)
    
fpr, tpr, thresholds = metrics.roc_curve(y_true, pred, pos_label = 0)

auc1 = metrics.auc(fpr, tpr)

reports = classification_report(y_true, pred, output_dict=True) 
classification_report_csv(report = reports, auc = auc1, check_folder = checkpoint_folder)

Epoch: 1


259it [00:42,  6.15it/s]


KeyboardInterrupt: 

447it [00:03, 145.12it/s]

462it [00:03, 145.86it/s]

...

1038it [00:07, 152.21it/s]

1056it [00:07, 157.86it/s]

1059it [00:07, 133.67it/s]

valid loss: 0.757203476219201 - valid acc: 91.78470254957507
Best acuracy: 0.9216241737488197 at epoch 62



/media/mountHDD2/linh/git/.env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/media/mountHDD2/linh/git/.env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/media/mountHDD2/linh/git/.env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# print(reports.keys())
# print(reports['macro avg'])

# Evaluation